<a href="https://colab.research.google.com/github/evroth/location-optimization/blob/main/Uber_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json


def get_ride_price(origin_latitude, origin_longitude, destination_latitude, destination_longitude):
    url = "https://www.uber.com/api/loadFEEstimates?localeCode=en"

    payload = json.dumps({
      "origin": {
        "latitude": origin_latitude,
        "longitude": origin_longitude
      },
      "destination": {
        "latitude": destination_latitude,
        "longitude": destination_longitude
      },
      "locale": "en"
    })
    headers = {
      'content-type': 'application/json',
      'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'x-csrf-token': 'x'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    result = [[x['vehicleViewDisplayName'], x['fareString']] for x in response.json()['data']['prices']]

    # I want to add something to just reference the UberX price but it isn't always in the same location every time

    return result

In [5]:
print(get_ride_price(51.5072178, -0.1275862, 51.4974948, -0.1356583))

[['UberX', '£8.87'], ['Assist', '£8.87'], ['Access', '£8.87'], ['Green', '£8.87'], ['Comfort', '£10.78'], ['UberXL', '£12.23'], ['Uber Pet', '£13.87'], ['Exec', '£14.91'], ['Lux', '£20.11']]


In [2]:
result = [['UberX', '£8.44'], ['Green', '£8.44'], ['Assist', '£8.45'], ['Access', '£8.45'], ['Comfort', '£10.25'], ['UberXL', '£11.68'], ['Uber Pet', '£13.43'], ['Exec', '£14.11'], ['Lux', '£19.02']]

In [ ]:
# figuring out how to reference
result[['UberX']]

In [4]:
print(get_ride_price(47.447277, -122.3039276, 47.61828319569383, -122.19681018164148))

[['WAV', '$49.05'], ['Connect', '$52.72'], ['UberX', '$55.99'], ['Uber Green', '$55.99'], ['Comfort', '$59.64'], ['Black Hourly', '$120.00'], ['Uber Pet', '$59.99'], ['Comfort Electric', '$65.61'], ['UberXL', '$70.00'], ['Black', '$80.18'], ['Black SUV', '$90.22']]
